In [ ]:
# Mounting the google colab drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

#comments_to_score
comments_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/jigsaw-toxic-severity-rating/comments_to_score.csv")

#vaidation_Data_set
val_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/jigsaw-toxic-severity-rating/validation_data.csv")

#train_dataset_from_first_Kaggle_competition
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/jigsaw-toxic-comment-classification-challenge/train.csv")

In [ ]:
comments_df.head()

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


In [ ]:
val_df.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [ ]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
#combine val_data_df into single comments list
less_toxic=val_df['less_toxic'].tolist()

In [ ]:
#less_toxic_comments_len
len(less_toxic)

30108

In [ ]:
#more_toxic_comments
more_toxic=val_df['more_toxic'].tolist()

#less_toxic_comments_len
len(more_toxic)

30108

In [ ]:
#all_comments
all_comments = less_toxic + more_toxic
len(all_comments)

60216

Following this kaggle user's notebook: https://www.kaggle.com/vitaleey/tfidf-ridge

In [ ]:
import nltk
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
#let's create category weights for each of the labels in Kaggle's 1st training datset

# Create a score that messures how much toxic is a comment
cat_weights = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_weights:
    train_df[category] = train_df[category] * cat_weights[category]

In [ ]:
train_df[train_df['severe_toxic']>0].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,0.32,1.5,0.16,0.0,0.64,0.0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,0.32,1.5,0.16,0.0,0.64,0.0
181,006e87872c8b370c,you are a stupid fuck \n\nand your mother's cu...,0.32,1.5,0.16,0.0,0.64,0.0
442,01208d2b76624130,Hi \n\nIm a fucking bitch.\n\n50.180.208.181,0.32,1.5,0.16,0.0,0.64,0.0
579,018663f910e0bfe6,What a motherfucking piece of crap those fuckh...,0.32,1.5,0.16,0.0,0.64,0.0


In [ ]:
#take the weighted sum of each category as the final toxicity score for each comment
train_df['score'] = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)

In [ ]:
#only keep the comment and score and save as train
train = train_df[['comment_text','score']]
train.head()

,comment_text,score
0,Explanation\nWhy the edits made under my usern...,0.0
1,D'aww! He matches this background colour I'm s...,0.0
2,"Hey man, I'm really not trying to edit war. It...",0.0
3,"""\nMore\nI can't make any real suggestions on ...",0.0
4,"You, sir, are my hero. Any chance you remember...",0.0


In [ ]:
train.score.value_counts()

0.00    143346
0.32      5666
1.12      3800
0.48      1758
2.62      1738
0.96      1215
4.12       385
0.16       317
0.64       301
1.82       290
1.98       204
0.80       181
2.46       164
1.50        76
5.62        31
2.14        31
3.32        21
2.30        20
3.48        10
3.96        10
1.66         5
3.96         1
4.82         1
Name: score, dtype: int64

Since Data is highly imbalanced, we need to undersample the comments that have no toxicity component

In [ ]:
#check the length of comments with some toxicity score
min_len = (train['score'] > 0).sum()  # len of toxic comments

In [ ]:
#undersample non-toxic comments
df_y0_undersample = train[train['score'] == 0].sample(n=min_len, random_state=201)  # take non toxic comments
df_train_new = pd.concat([train[train['score'] > 0], df_y0_undersample])  # make new df
df_train_new.head()

,comment_text,score
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,2.62
12,Hey... what is it..\n@ | talk .\nWhat is it......,0.32
16,"Bye! \n\nDon't look, come or think of comming ...",0.32
42,You are gay or antisemmitian? \n\nArchangel WH...,2.62
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1.12


In [ ]:
#check value_counts of score now
df_train_new['score'].value_counts()

0.00    16225
0.32     5666
1.12     3800
0.48     1758
2.62     1738
0.96     1215
4.12      385
0.16      317
0.64      301
1.82      290
1.98      204
0.80      181
2.46      164
1.50       76
5.62       31
2.14       31
3.32       21
2.30       20
3.96       10
3.48       10
1.66        5
4.82        1
3.96        1
Name: score, dtype: int64

### Text preprocessing

In [ ]:
#function for text cleaning

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

import re
def text_cleaning(text):
    forbidden_words = set(stopwords.words('english'))
    if text:
        text = ' '.join(text.split('.'))
        text = re.sub('\/',' ',text)
        text = re.sub(r'\\',' ',text)
        text = re.sub(r'((http)\S+)','',text)
        text = re.sub('\s+', ' ', text)  # will remove more than one whitespace character
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        
        """ Replaces url address with "url" """
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
        text = re.sub(r'#([^\s]+)', r'\1', text)
        
        #remove multiple toxic words
        text = re.sub(r'(fuckfuck)','fuck fuck ',text)
        text = re.sub(r'(f+)( *)([u|*]+)( *)([c|*]+)( *)(k)+','fuck',text)
        text = re.sub(r'(s+ *h+ *i+ *t+)','shit',text)
        text = re.sub(r'([a|@][$|s][s|$])','ass',text)
        text = re.sub(r'(\bfuk\b)','fuck',text)
        text = re.sub(r'(\bpennnis\b)','penis',text)
        text = re.sub(r'(\bfggt\b)','faggot',text)
        text = re.sub(r'(\bfag\b)','faggot',text)
        
        #replaces abreviations
        text = re.sub(r"what's", "what is ",text)    
        text = re.sub(r"\'ve", " have ",text)
        text = re.sub(r"can't", "cannot ",text)
        text = re.sub(r"n't", " not ",text)
        text = re.sub(r"i'm", "i am ",text)
        text = re.sub(r"\'re", " are ",text)
        text = re.sub(r"\'d", " would ",text)
        text = re.sub(r"\'ll", " will ",text)
        text = re.sub(r"\'scuse", " excuse ",text)
        text = re.sub(r"\'s"," ",text)
        
        #remove unicode strings
        """ Removes unicode strings like "\u002c" and "x96" """
        text = re.sub(r'(\\u[0-9A-Fa-f]+)',r' ', text)       
        text = re.sub(r'[^\x00-\x7f]',r' ',text)
        
        """ Replaces "@user" with "atUser" """
        text = re.sub('@[^\s]+','atUser',text)
        
        text = [word for word in text.split() if word not in forbidden_words]
        
        
        
        #lemmatize words
        lemmatizer = WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        return text
    return []

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Text cleaning

In [ ]:
#clean the comments

df_train_new['comment_text'] = df_train_new['comment_text'].apply(lambda x: ' '.join(text_cleaning(x)))

X_clean=df_train_new['comment_text'].tolist()

In [ ]:
len(X_clean)

32450

In [ ]:
len(df_train_new)

32450

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X_clean)

In [ ]:
vectorizer.vocabulary_

{'cocksucker': 9282,
 'piss': 37610,
 'around': 2898,
 'work': 55422,
 'hey': 22478,
 'talk': 48850,
 'exclusive': 16872,
 'group': 21110,
 'wp': 55509,
 'taliban': 48846,
 'good': 20623,
 'destroying': 12960,
 'self': 44176,
 'appointed': 2543,
 'purist': 39606,
 'gang': 19695,
 'one': 35188,
 'asks': 3164,
 'question': 39911,
 'abt': 222,
 'anti': 2258,
 'social': 46061,
 'destructive': 12965,
 'non': 34153,
 'contribution': 10463,
 'ask': 3154,
 'sityush': 45489,
 'clean': 8999,
 'behavior': 4841,
 'issue': 25537,
 'nonsensical': 34196,
 'warning': 54055,
 'bye': 7144,
 'look': 29234,
 'come': 9539,
 'think': 49813,
 'comming': 9635,
 'back': 4001,
 'tosser': 50540,
 'gay': 19829,
 'antisemmitian': 2301,
 'archangel': 2708,
 'white': 54586,
 'tiger': 50077,
 'meow': 31106,
 'greetingshhh': 20987,
 'uh': 51538,
 'two': 51397,
 'way': 54187,
 'erased': 16279,
 'comment': 9614,
 'ww': 55662,
 'holocaust': 22866,
 'brutally': 6719,
 'slaying': 45679,
 'jew': 26031,
 'gypsy': 21394,
 'sl

In [ ]:
#define target variable
y = df_train_new['score']

# from sklearn.linear_model import Ridge
# from sklearn.model_selection import cross_validate
# clf = Ridge(alpha=0.7)

# scores = cross_validate(clf, X_vec, y, cv=5,scoring=('r2'), return_train_score=True)

# print(np.mean(scores['test_score']))
# clf.fit(X_vec, y)

In [ ]:
# evaluate an ridge regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge

# define model
clf = Ridge(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(clf, X_vec, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: 0.338 (0.007)


In [ ]:
# fit model
clf.fit(X_vec, y)

Ridge()

In [ ]:
X_vec.shape

(32450, 56572)

In [ ]:
feature_wts = sorted(list(zip(vectorizer.get_feature_names(), np.round(clf.coef_,2) )), 
                     key = lambda x:x[1], reverse=True)
print(feature_wts[0:30])

[('nigger', 2.99), ('nigga', 2.86), ('homosexual', 2.5), ('niggaz', 2.45), ('die', 2.3), ('kill', 2.23), ('jew', 2.1), ('gay', 2.07), ('fuck', 2.03), ('faggot', 1.98), ('fucking', 1.96), ('asyriac', 1.93), ('fucker', 1.85), ('nl', 1.84), ('rape', 1.83), ('ancestryfuck', 1.81), ('hooker', 1.81), ('filthy', 1.8), ('malaysian', 1.76), ('motherfucker', 1.72), ('fagot', 1.71), ('fuckin', 1.71), ('hawking', 1.7), ('negro', 1.7), ('homo', 1.68), ('niggerballs', 1.62), ('nig', 1.59), ('bitch', 1.57), ('nazi', 1.53), ('jujube', 1.51)]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Validating the results

In [ ]:
#getting scores for less toxic and more toxic comments
less_toxic_cleaned = [' '.join(text_cleaning(line)) for line in less_toxic]
more_toxic_cleaned = [' '.join(text_cleaning(line)) for line in more_toxic]

In [ ]:
#get tf-idf vector
less_toxic_vec = vectorizer.transform(less_toxic_cleaned)
more_toxic_vec =vectorizer.transform(more_toxic_cleaned)

In [ ]:
#get predicted score
y_pred_less = clf.predict(less_toxic_vec)
y_pred_more = clf.predict(more_toxic_vec)

In [ ]:
#adding the scores to the validation dataset
val_df['less_toxic_score'] = y_pred_less
val_df['more_toxic_score'] = y_pred_more

In [ ]:
val_df.head()

,worker,less_toxic,more_toxic,less_toxic_score,more_toxic_score
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,0.572144,0.477789
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,-0.082579,0.677213
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0.196221,0.369251
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,0.312093,1.567696
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",0.876000,1.467101


In [ ]:
#check if our model predicts the toxicity correctly
def function(x):
    if x.less_toxic_score < x.more_toxic_score:
        return 1
    else:
        return 0
    
val_df["result"] = val_df[["less_toxic_score", "more_toxic_score"]].apply(function, axis=1)

In [ ]:
val_df['result'].value_counts(normalize=True)

1    0.670353
0    0.329647
Name: result, dtype: float64

In [ ]:
# 1    0.671217
# 0    0.328783

# results after second data cleaning and not removing stopwords
# 1    0.670586
# 0    0.329414

# 3rd try
# 1    0.670885
# 0    0.329115

#4th try with changes in text cleaning alpha=0.5
# 1    0.680517
# 0    0.319483

#5th try alpha=0.4
# 1    0.678923
# 0    0.321077

#6th try alpha=0.6
# 1    0.681679
# 0    0.318321

#7th try alpha=0.7
# 1    0.681679
# 0    0.318321

#8th try alpha=1
# 1    0.672512
# 0    0.327488

### Hyper parameter optimization for Ridge

In [ ]:
# grid search hyperparameters for ridge regression
from numpy import arange
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge


# define model
model = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.3)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
# perform the search
results = search.fit(X_vec, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -0.347
Config: {'alpha': 0.8999999999999999}


### Preparing data for submission

In [ ]:
comments2score = pd.read_csv("jigsaw-toxic-severity-rating/comments_to_score.csv")
#getting scores for less toxic and more toxic comments
comm2score = [my_cleaner(line) for line in comments2score['text']]

FileNotFoundError: ignored

In [ ]:
comm2score[:2]

In [ ]:
comm_vec = vectorizer.transform(comm2score)

In [ ]:
comm_vec.shape

In [ ]:
#get predicted score
y_comm_vec = clf.predict(comm_vec)

In [ ]:
y_comm_vec

In [ ]:
comment_id = comments2score['comment_id'].tolist()
score = list(y_comm_vec)

In [ ]:
df_sub = pd.DataFrame({"comment_id":comment_id, 'score':score})

In [ ]:
df_sub.to_csv("submission4_tfidf.csv", index=False)